<a href="https://colab.research.google.com/github/shanikairoshi/Communication-Efficient-DUQFL/blob/main/main_v2_1.3_gamma_0.5_withTele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%capture
!pip install genomic-benchmarks
!pip install qiskit qiskit_machine_learning qiskit_algorithms qiskit-aer




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
from pathlib import Path
PROJ = Path.cwd() / "tduqfl_Project_AGG"
if str(PROJ) not in sys.path:
    sys.path.insert(0, str(PROJ))
import sys
sys.path.append('/content/drive/MyDrive/Teleportation/tduqfl_Project_AGG/tDuQFL_Project')
# ─── 5. Assemble filenames for each artifact ─────────────────────────────────
#drive_root = "/content/drive/MyDrive/Teleportation/tduqfl_Project_AGG/tDuQFL_Project/"

In [4]:
from common.imports import *
from configs.dataset_genome_iid import *     # swap to other configs as needed
from io_utils.naming import stamp_now, flags, build_param_str, make_filenames

start_str, date_str = stamp_now()
teleport_pl, noise_pl = flags(use_teleportation, use_noise)
param_str = build_param_str(num_clients, num_federated_layers, num_deep_unfolding_iterations,
                            initial_learning_rate, initial_perturbation)

best_client_csv_file, global_csv_file, local_csv_file, validation_csv_file = make_filenames(
    drive_root, dataset_name, split_type, date_str, teleport_pl, noise_pl, param_str
)
from io_utils.csv_logger import init_local_csv, init_best_csv, init_validation_csv

# Create folders + write headers
init_best_csv(best_client_csv_file)

local_headers = [
    "Federated Round", "Client Number", "Iteration",
    "Objective Function Value", "Training Accuracy", "Test Accuracy",
    "Learning Rate", "Perturbation"
]
init_local_csv(local_csv_file, local_headers)

init_validation_csv(validation_csv_file)

# Do NOT pre-init global_csv_file here because your save_accuracies_to_csv()
# already writes the header each time it runs (in 'w' mode).

/usr/local/lib/python3.12/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Qiskit: 1.4.4
qiskit_aer available?: True


Load and Split data

In [5]:
from data.preprocess_genome import load_and_prepare_dataset
from data.splitters import split_dataset_for_epochs
from configs.base_config import (
    num_clients, num_epochs, samples_per_epoch, split_type,
    global_seed,client_hparams_csv_file
)

np_train_data, np_test_data = load_and_prepare_dataset(word_size, global_seed)

# 2) Compute feasible epoch capacity and cap both epochs and rounds
N_train = len(np_train_data)
train_capacity = N_train // (num_clients * samples_per_epoch)
num_epochs_eff = max(1, min(num_epochs, train_capacity))

if train_capacity == 0:
    raise ValueError(
        f"Not enough training samples ({N_train}) for "
        f"{num_clients=} × {samples_per_epoch=} per epoch. "
        "Reduce samples_per_epoch or num_clients, or enable resampling."
    )

num_federated_layers_eff = min(num_federated_layers, num_epochs_eff)

# Build clients
if split_type.lower() == "iid":
    from data.splitters import split_dataset_for_epochs
    clients = split_dataset_for_epochs(
        num_clients=num_clients,
        num_epochs=num_epochs_eff,             # or num_epochs
        train_data=np_train_data,
        test_data=np_test_data,
        samples_per_epoch=samples_per_epoch,
    )
elif split_type.lower() in {"noniid", "non-iid", "non_iid"}:
    from data.noniid import make_non_iid_clients
    clients = make_non_iid_clients(
        train_data=np_train_data,
        test_data=np_test_data,
        num_clients=num_clients,
        num_epochs=num_epochs_eff,             # or num_epochs
        samples_per_epoch=samples_per_epoch,
        non_iid_ratio=0.8,                     # tune as needed
        quantity_variation=0.5,                # tune as needed
        seed=global_seed,
        plot=True
    )
else:
    raise ValueError(f"Unknown split_type: {split_type}")

'''
clients = split_dataset_for_epochs(
    num_clients=num_clients, num_epochs=num_epochs,
    train_data=np_train_data, test_data=np_test_data,
    samples_per_epoch=samples_per_epoch
)
'''
# validation/tables
test_sequences = np.array([d["sequence"] for d in np_test_data])
test_labels    = np.array([d["label"]    for d in np_test_data])
X_val, y_val   = test_sequences, test_labels

# derive num_features once
if clients and clients[0].data and clients[0].data[0]:
    num_features = clients[0].data[0][0]['sequence'].shape[0]
else:
    raise RuntimeError("Empty client data – check splitting indices.")


In [6]:
# Infer num_features from the first available sample in clients
def infer_num_features_from_clients(clients):
    for c in clients:
        for epoch_data in c.data:              # list of samples for that epoch
            if not epoch_data:
                continue
            sample = epoch_data[0]
            if "sequence" in sample:           # your Genome pipeline
                arr = np.asarray(sample["sequence"])
                return int(arr.size)
            if "features" in sample:           # some other pipelines
                arr = np.asarray(sample["features"])
                return int(arr.size)
            if "image" in sample:              # e.g., MNIST before flatten
                arr = np.asarray(sample["image"]).reshape(-1)
                return int(arr.size)
            # add any other key you use
    raise RuntimeError("Could not infer num_features: no samples found.")

num_features = infer_num_features_from_clients(clients)
print(f"[info] num_features = {num_features}")


[info] num_features = 5


run federated loop and plot

In [ ]:
from training.loop import run_federated_training
from configs.base_config import use_teleportation as CFG_TEL, noise_preset, shots_used,aggregation,client_hparams_csv_file
from training.metrics import metrics_init, metrics_log_round, metrics_finalize, compute_auc,metrics_summarize
from viz.plots import plot_accuracy_curve, plot_val_loss, plot_time_per_round, plot_fidelity_vs_delta_acc, plot_beta_hist, plot_client_fairness_last_round
# Initialize metrics store once
metrics_store = metrics_init(
    log_path=os.path.join(drive_root, "teleport_metrics_Perturb_shrink.csv")
)

#new
from ml import optimizers as mlopt
from configs.base_config import drive_root
import os

mlopt.meta_trace_enable(
    path=os.path.join(drive_root, "meta_trace.csv"),  # or None to skip CSV
    every=5                                           # print every 5 callbacks
)

###########
global_acc, clients_train, clients_test, round_times, val_losses, info_last = run_federated_training(
    clients=clients,
    num_federated_layers=num_federated_layers,
    num_deep_unfolding_iterations=num_deep_unfolding_iterations,
    initial_learning_rate=initial_learning_rate,
    initial_perturbation=initial_perturbation,
    num_features=num_features,
    best_client_csv_file=best_client_csv_file,
    global_csv_file=global_csv_file,
    local_csv_file=local_csv_file,
    validation_csv_file=validation_csv_file,
    test_sequences=test_sequences,
    test_labels=test_labels,
    X_val=X_val,
    y_val=y_val,
    use_teleportation=CFG_TEL,          # ← important
    noise_preset=noise_preset,
    shots_used=shots_used,
    metrics=metrics_store,   # <-- pass it in
    aggregation=aggregation,
    client_hparams_csv_file=client_hparams_csv_file,   # <-- NEW# <--- switch here
)

rows_np = metrics_finalize(metrics_store)   # if you need the in-memory array
#summary = metrics_summarize(metrics_store)  # prints a concise summary, returns a dict

# quick visuals
rounds = list(range(len(global_acc)))
plot_accuracy_curve(rounds, global_acc, label="Global accuracy (DT-DUQFL)")
plot_val_loss(rounds, val_losses, label="Central validation loss")
plot_time_per_round(rounds, round_times)

if info_last is not None:
    # this uses "last" round's info; in your logger you kept per-round arrays; adapt if needed
    pass


Training Progress:   0%|          | 0/10 [00:00<?, ?it/s]

[Round 0] Teleportation ON | Aggregation=best
[round 0 | client 0] seed LR=0.1300000000 (prev=0.1300000000), seed PERT=0.1300000000 (prev=0.1300000000), gamma=0.50


Deep Unfolding Iteration 1/5
[meta] cb#005 loss=0.477515 step=0.05831 g_raw=+0.025 g_sm=+0.003 acc=1 | LR→0.130260 PERT→0.130000 (scale=0.04)
[meta] cb#010 loss=0.475036 step=0.07344 g_raw=+0.033 g_sm=+0.004 acc=1 | LR→0.130521 PERT→0.130000 (scale=0.04)
[meta] cb#015 loss=0.474616 step=0.005678 g_raw=+0.003 g_sm=+0.004 acc=1 | LR→0.130783 PERT→0.130000 (scale=0.04)
[meta] cb#020 loss=0.469893 step=0.06736 g_raw=+0.027 g_sm=+0.007 acc=1 | LR→0.131045 PERT→0.130000 (scale=0.04)
[meta] cb#025 loss=0.463747 step=0.03181 g_raw=+0.015 g_sm=+0.010 acc=1 | LR→0.131307 PERT→0.130001 (scale=0.04)
[client 0 | unfold 1] LR_used=0.1300000000, PERT_used=0.1300000000 → LR_next=0.1313071330, PERT_next=0.1300006052
[meta-summary] callbacks=25 g_raw_mean=+0.013 g_sm_mean=+0.005 acc_ratio=1.00 | LR 0.1300000000→0.1313071330 PERT 0.130000000

Training Progress:  10%|█         | 1/10 [16:12<2:25:48, 972.05s/it]

[Round   0] acc_g=0.797 (μ=0.658, σ=0.110, FG=0.237) | t=953.720s, val=0.545 | TEL β=0.108, Fid μ=0.727, σ=0.180, p50=0.764, p90=0.944 | Lat μ=NA, σ=NA, p90=NA | Inst μ=NA, σ=NA | Δ per-dim=0.254, ||Δ||₂=5.074, max|Δ|=1.826 | noise=med, shots=256
[Round 1] Teleportation ON | Aggregation=best
[round 1 | client 0] seed LR=0.1333362940 (prev=0.1366725879), seed PERT=0.1300034936 (prev=0.1300069872), gamma=0.50


Deep Unfolding Iteration 1/5
[meta] cb#005 loss=0.480822 step=0.01185 g_raw=+0.005 g_sm=+0.005 acc=1 | LR→0.133603 PERT→0.130004 (scale=0.04)
[meta] cb#010 loss=0.475269 step=0.05928 g_raw=+0.026 g_sm=+0.008 acc=1 | LR→0.133871 PERT→0.130004 (scale=0.04)
[meta] cb#015 loss=0.469363 step=0.05639 g_raw=+0.023 g_sm=+0.011 acc=1 | LR→0.134139 PERT→0.130004 (scale=0.04)
[meta] cb#020 loss=0.467710 step=0.02279 g_raw=+0.010 g_sm=+0.011 acc=1 | LR→0.134408 PERT→0.130004 (scale=0.04)
[meta] cb#025 loss=0.465113 step=0.04369 g_raw=+0.018 g_sm=+0.011 acc=1 | LR→0.134677 PERT→0.130005 (scale

Training Progress:  20%|██        | 2/10 [32:20<2:09:20, 970.12s/it]

...

[Round   3] acc_g=0.816 (μ=0.721, σ=0.098, FG=0.210) | t=969.048s, val=0.538 | TEL β=0.133, Fid μ=0.733, σ=0.138, p50=0.725, p90=0.893 | Lat μ=NA, σ=NA, p90=NA | Inst μ=NA, σ=NA | Δ per-dim=0.247, ||Δ||₂=4.947, max|Δ|=1.788 | noise=med, shots=256
[Round 4] Teleportation ON | Aggregation=best
[round 4 | client 0] seed LR=0.1364963052 (prev=0.1429926103), seed PERT=0.1300063048 (prev=0.1300126097), gamma=0.50


Deep Unfolding Iteration 1/5
[meta] cb#005 loss=0.486471 step=0.03074 g_raw=+0.007 g_sm=+0.002 acc=1 | LR→0.136770 PERT→0.130006 (scale=0.04)
[meta] cb#010 loss=0.485485 step=0.007927 g_raw=+0.002 g_sm=+0.003 acc=1 | LR→0.137044 PERT→0.130006 (scale=0.04)
[meta] cb#015 loss=0.482506 step=0.07096 g_raw=+0.027 g_sm=+0.005 acc=1 | LR→0.137318 PERT→0.130007 (scale=0.04)
[meta] cb#020 loss=0.479449 step=0.05509 g_raw=+0.022 g_sm=+0.008 acc=1 | LR→0.137593 PERT→0.130007 (scale=0.04)
[meta] cb#025 loss=0.473512 step=0.09639 g_raw=+0.040 g_sm=+0.010 acc=1 | LR→0.137869 PERT→0.130007 (scal

Training Progress:  50%|█████     | 5/10 [1:21:47<1:21:38, 979.75s/it]

[Round   4] acc_g=0.788 (μ=0.731, σ=0.065, FG=0.142) | t=944.224s, val=0.544 | TEL β=0.133, Fid μ=0.733, σ=0.125, p50=0.746, p90=0.877 | Lat μ=NA, σ=NA, p90=NA | Inst μ=NA, σ=NA | Δ per-dim=0.246, ||Δ||₂=4.929, max|Δ|=1.636 | noise=med, shots=256
[Round 5] Teleportation ON | Aggregation=best
[round 5 | client 0] seed LR=0.1367507488 (prev=0.1435014976), seed PERT=0.1300062678 (prev=0.1300125356), gamma=0.50


Deep Unfolding Iteration 1/5
[meta] cb#005 loss=0.532150 step=0.007344 g_raw=+0.002 g_sm=+0.001 acc=1 | LR→0.137025 PERT→0.130006 (scale=0.04)
[meta] cb#010 loss=0.529861 step=0.03712 g_raw=+0.016 g_sm=+0.003 acc=1 | LR→0.137299 PERT→0.130006 (scale=0.04)
[meta] cb#015 loss=0.528492 step=0.04976 g_raw=+0.021 g_sm=+0.004 acc=1 | LR→0.137574 PERT→0.130006 (scale=0.04)
[meta] cb#020 loss=0.526233 step=0.04096 g_raw=+0.018 g_sm=+0.006 acc=1 | LR→0.137849 PERT→0.130007 (scale=0.04)
[meta] cb#025 loss=0.524681 step=0.0006402 g_raw=-0.003 g_sm=+0.007 acc=1 | LR→0.138126 PERT→0.130007 (sc

Training Progress:  60%|██████    | 6/10 [1:37:47<1:04:52, 973.08s/it]

[Round   5] acc_g=0.750 (μ=0.729, σ=0.064, FG=0.142) | t=942.729s, val=0.540 | TEL β=0.109, Fid μ=0.727, σ=0.133, p50=0.743, p90=0.842 | Lat μ=NA, σ=NA, p90=NA | Inst μ=NA, σ=NA | Δ per-dim=0.250, ||Δ||₂=5.002, max|Δ|=1.719 | noise=med, shots=256
[Round 6] Teleportation ON | Aggregation=best
[round 6 | client 0] seed LR=0.1368847885 (prev=0.1437695769), seed PERT=0.1300065103 (prev=0.1300130206), gamma=0.50


Deep Unfolding Iteration 1/5
[meta] cb#005 loss=0.517816 step=0.06148 g_raw=+0.025 g_sm=+0.005 acc=1 | LR→0.137159 PERT→0.130007 (scale=0.04)
[meta] cb#010 loss=0.504726 step=0.06993 g_raw=+0.026 g_sm=+0.010 acc=1 | LR→0.137434 PERT→0.130007 (scale=0.04)
[meta] cb#015 loss=0.493154 step=0.04565 g_raw=+0.019 g_sm=+0.014 acc=1 | LR→0.137709 PERT→0.130007 (scale=0.04)
[meta] cb#020 loss=0.477497 step=0.1042 g_raw=+0.042 g_sm=+0.019 acc=1 | LR→0.137985 PERT→0.130008 (scale=0.04)
[meta] cb#025 loss=0.467685 step=0.03153 g_raw=+0.015 g_sm=+0.020 acc=1 | LR→0.138262 PERT→0.130008 (scale=